<a href="https://colab.research.google.com/github/MertIV/hello-world/blob/master/Amazon_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests # required for HTTP requests: pip install requests
from bs4 import BeautifulSoup  # required for HTML and XML parsing: pip install beautifulsoup4
import pandas as pd # required for getting the data in dataframes : pip install pandas
import time # to time the requests
from multiprocessing import Manager,Process, Queue, Pool
import threading
import sys

In [0]:
proxies = { # define the proxies which you want to use
  'http': 'http://195.22.121.13:443',
  'https': 'http://195.22.121.13:443',
}
startTime = time.time()
qcount = 0 # the count in queue used to track the elements in queue
products=[] # List to store name of the product
prices=[] # List to store price of the product
ratings=[] # List to store ratings of the product
no_pages = 9 # no of pages to scrape in the website (provide it via arguments)

In [0]:
def get_data(pageNo,q):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64;     x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate",     "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    r = requests.get("https://www.amazon.com/s?k=laptops&page="+str(pageNo), headers=headers)#,proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)

    for d in soup.findAll('div', attrs={'class':'sg-col-4-of-12 sg-col-8-of-16 sg-col-16-of-24 sg-col-12-of-20 sg-col-24-of-32 sg-col sg-col-28-of-36 sg-col-20-of-28'}):
            name = d.find('span', attrs={'class':'a-size-medium a-color-base a-text-normal'})
            price = d.find('span', attrs={'class':'a-offscreen'})
            rating = d.find('span', attrs={'class':'a-icon-alt'})
            all=[]
    if name is not None:
      all.append(name.text)
    else:
      all.append("unknown-product")
    if price is not None:
      all.append(price.text)
    else:
      all.append('$0')
    if rating is not None:
    #print(rating.text)
      all.append(rating.text)
    else:
      all.append('-1')
      q.put(all)
      print("---------------------------------------------------------------")
    results = []

In [12]:
if __name__ == "__main__":
    m = Manager()
    q = m.Queue()
    p = {}
    if sys.argv[1] in ['t', 'p']:
        for i in range(1,no_pages):
            if sys.argv[1] in ['t']:
                print("starting thread: ",i)
                p[i] = threading.Thread(target=get_data, args=(i,q))
                p[i].start()
            elif sys.argv[1] in ['p']:
                print("starting process: ",i)
                p[i] = Process(target=get_data, args=(i,q))
                p[i].start()
        for i in range(1,no_pages): # join all the threads/processes
            p[i].join()
    else:
        pool_tuple = [(x,q) for x in range(1,no_pages)]
        with Pool(processes=8) as pool:
            print("in pool")
            results = pool.starmap(get_data, pool_tuple)
        print(results)
    while q.empty() is not True:
            qcount = qcount+1
            queue_top = q.get()
            products.append(queue_top[0])
            prices.append(queue_top[1])
            ratings.append(queue_top[2])

    print("total time taken: ", str(time.time()-startTime), " qcount: ", qcount)
    df = pd.DataFrame({'Product Name':products, 'Price':prices, 'Ratings':ratings})
    df.to_csv('products.csv', index=False, encoding='utf-8')

in pool
---------------------------------------------------------------
---------------------------------------------------------------
---------------------------------------------------------------
---------------------------------------------------------------
---------------------------------------------------------------
[None, None, None, None, None, None, None, None]
total time taken:  498.8915147781372  qcount:  5


In [0]:
while q.empty() is not True:
        qcount = qcount+1
        queue_top = q.get()
        products.append(queue_top[0])
        prices.append(queue_top[1])
        ratings.append(queue_top[2])

In [14]:
df

,Product Name,Price,Ratings
0,"ASUS VivoBook Flip 2020 11.6"" Touch Screen 2-i...",$306.99,-1
1,"2020 ASUS VivoBook 15 15.6"" FHD Laptop Compute...",$429.00,-1
2,"ASUS VivoBook Flip 2020 11.6"" Touch Screen 2-i...",$306.99,-1
3,"2020 ASUS VivoBook 15 15.6"" FHD Laptop Compute...",$429.00,-1
4,"2020 ASUS VivoBook 15 15.6"" FHD Laptop Compute...",$429.00,-1
